In [473]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")



from analysis_helpers import poisson_waiting_function
from subdivide_severity import subdivide_severity
from stochastic_simulation import StochasticSimulation
import unittest


In [474]:
# copy from base_params
pct_self_reports_severe = 1

daily_self_report_severe = 0.8
daily_self_report_mild = 0


avg_infectious_window = 4 + pct_self_reports_severe * (1 / daily_self_report_severe) 
if daily_self_report_mild != 0:
    avg_infectious_window += (1 - pct_self_reports_severe) * (1 / daily_self_report_mild)
pre_reopen_population = 2500
pre_reopen_daily_contacts = 7


prob_severity_given_age = np.array([[0.05, 0.94, 0.01, 0],\
                                    [0.04, 0.83, 0.10, 0.03],\
                                    [0.04, 0.79, 0.10, 0.07],\
                                    [0.04, 0.73, 0.13, 0.10],\
                                    [0.03, 0.57, 0.2, 0.2]])

prob_infection = np.array([0.018, 0.022, 0.029, 0.042, 0.042])
prob_age = np.array([0, 0.6534332, 0.3221892, 0.0209788, 0.0033988])

In [475]:
# test parameters
test_params = {
    'max_time_exposed': 4,
    'exposed_time_function': poisson_waiting_function(max_time=4, mean_time=2),
    
    'max_time_pre_ID': 4,
    'pre_ID_time_function': poisson_waiting_function(max_time=4, mean_time=0),
    
    'max_time_ID': 8,
    'ID_time_function': poisson_waiting_function(max_time=8, mean_time=3),
    
    'max_time_SyID_mild': 14,
    'SyID_mild_time_function': poisson_waiting_function(max_time=14, mean_time=10),
    
    'max_time_SyID_severe': 14,
    'SyID_severe_time_function': poisson_waiting_function(max_time=14, mean_time=10),
    
    'sample_QI_exit_function': (lambda n: np.random.binomial(n, 0.05)),
    'sample_QS_exit_function': (lambda n: np.random.binomial(n, 0.3)),
    
    'exposed_infection_p': 0.026,
    'expected_contacts_per_day': pre_reopen_daily_contacts,
    
    'mild_severity_levels': 1,
    'severity_prevalence': subdivide_severity(prob_severity_given_age, prob_infection, prob_age),
    'mild_symptoms_daily_self_report_p': daily_self_report_mild,
    'severe_symptoms_daily_self_report_p': daily_self_report_severe,
    
    'days_between_tests': 300,
    'test_population_fraction': 0,
    
    'test_protocol_QFNR': 0.1,
    'test_protocol_QFPR': 0.005,
    
    'perform_contact_tracing': True,
    'contact_tracing_constant': 0.5,
    'contact_tracing_delay': 1,
    'contact_trace_infectious_window': avg_infectious_window,
    
    'pre_ID_state': 'detectable',
    
    'population_size': 100,
    'initial_E_count': 0,
    'initial_pre_ID_count': 0,
    'initial_ID_count': 0,
    'initial_ID_prevalence': 0.001,
    'initial_SyID_mild_count': 0,
    'initial_SyID_severe_count': 0
}

In [476]:
# 99% confidence interavl test

class Testing(unittest.TestCase):
    
    longMessage = False
    
    def test_estimate_infection(self):
        
        # mean and var from replications
        mean1 = 2.192
        var1 = 4.765135999999998  #var = mean(abs(x - x.mean())**2)
        
        
        simulation = np.zeros(100)
        for n in range(100):
            sim = StochasticSimulation(test_params)
            for _ in range(100):
                sim.step()
            simulation[n] = 100 - sim.S - sim.QS   
            
        # mean and varr from simulations
        mean2 = np.mean(simulation) 
        var2 = np.var(simulation,ddof=1)
        
        #2.33 99%
        lb = (mean1-mean2) - 2.33* math.sqrt(var1/1000.0+var2/100.0)
        ub = (mean1-mean2) + 2.33* math.sqrt(var1/1000.0+var2/100.0)
        self.assertTrue(lb <= 0 <= ub)
        
        print mean2, var2, lb, ub

In [477]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_estimate_infection (__main__.Testing) ... 

1.96 2.7054545454545456 -0.1836270788201626 0.647627078820163


ok

----------------------------------------------------------------------
Ran 1 test in 85.485s

OK


In [ ]:
# some variable values test

In [ ]:
class Testing(unittest.TestCase):
    
    # initial parameters
    def setUp(self):
        np.random.seed(0)
        self.sim = StochasticSimulation(test_params)

    def test_e(self):
        self.assertEqual(self.sim.E.tolist(), np.array([0, 0, 0, 0]).tolist())
    def test_pre_id(self):
        self.assertEqual(self.sim.pre_ID.tolist(), np.array([0, 0, 0, 0]).tolist())
    def test_id(self):
        self.assertEqual(self.sim.ID.tolist(), np.array([0, 0, 1, 0, 0, 0, 0, 0]).tolist()) 
    def test_syid_mild(self):
        self.assertEqual(self.sim.SyID_mild.tolist(), np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]).tolist()) 
    def test_syid_severe(self):
        self.assertEqual(self.sim.SyID_severe.tolist(), np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]).tolist())
           
    # parameters after calling step()
    def test_step_id(self):
        self.sim.step()
        self.assertEqual(self.sim.ID.tolist(), np.array([0, 1, 0, 0, 0, 0, 0, 0]).tolist())
    
    def test_step_qi(self):
        self.sim.step()
        self.assertEqual(self.sim.QI, 0)
        
    def test_step_qi_mild(self):
        self.sim.step()
        self.assertEqual(self.sim.QI_mild, 0)
    
    def test_step_qi_severe(self):
        self.sim.step()
        self.assertEqual(self.sim.QI_severe, 0)
        
    def test_step_r(self):
        self.sim.step()
        self.assertEqual(self.sim.R, 0)
        
    def test_step_r_mild(self):
        self.sim.step()
        self.assertEqual(self.sim.R_mild, 0)
        
    def test_step_r_severe(self):
        self.sim.step()
        self.assertEqual(self.sim.R_severe, 0)
        
    def test_step_qs(self):
        self.sim.step()
        self.assertEqual(self.sim.QS, 0)
        
    def test_step_s(self):
        self.sim.step()
        self.assertEqual(self.sim.S, 99)

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
# 1000 replications to get an estimate

In [ ]:
import numpy as np
from test_params import test_params
from stochastic_simulation import StochasticSimulation

In [465]:
numSeed = 1000

infected = np.zeros(numSeed)

np.random.seed(0)

for n in range(numSeed):
    sim = StochasticSimulation(test_params)
        
    # 100-day estimate
    for s in range(100):
        sim.step()
        
    infected[n] = 100 - sim.S - sim.QS
    
print np.mean(infected) 
print np.var(infected)

2.192
4.765135999999998
